# Programming for Data Science and Artificial Intelligence

## Classification - Gradient Boosting

### Readings:
- [GERON] Ch7
- [VANDER] Ch5
- [HASTIE] Ch16
- https://scikit-learn.org/stable/modules/ensemble.html

## Gradient Boosting

Another popular one is Gradient Boosting.  Similar to AdaBoost, Gradient Boosting works by adding sequential predictors.  However, instead of adding **weights**, this method tries to fit the new predictor to the **residual errors** made by the previous predictor.    The hypothesis function of gradient boosting is as follows:

$$
H(x) = h_0(x) + \alpha_1h_1(x) + \cdots + \alpha_sh_s(x)
$$

Although they look similar, notice that no alpha is applied to the first predictor.  In addition, each alpha is the same, as opposed to voting power in AdaBoost.  Typically, similar to AdaBoost, decision trees are used for each $h_i(x)$ but are not limited to stump.  In practice, min_leaves are set to around 8 to 32.

Since gradient boosting actually originate from additive linear regression, we shall first talk about **gradient boosting for regression**.  Also assume that we are using **regression trees** for our regressors.

### Gradient Boosting for Regression

Firstly, let's look at the following equation where $h_0(x)$ is our first predictor and we would like to minimize the residual as follows:

$$h_0(x) + residual_0 = y $$
$$ residual_0 =  y - h_0(x) $$

That is, we would $y$ to be as close as $h_0(x)$ such that residual is 0

$$ y = h_0(x) $$

The question is that is it possible to add the second predictor $h_1(x)$ such that the residual is further reduced

$$ y = h_0(x) + h_1(x) $$

This equation can be written as:

$$h_1(x) = y - h_0(x) $$

This equation informs us that if we can find a subsequent predictor that can best fit the "residual" (i.e. $y - h_0(x)$), then we can improve the accuracy.

**How is this related to gradient descent?**

Well, firstly, here is our loss function for regression:

$$J = \frac{1}{2}(y - h(x))^2$$

And here, we want to minimize $J$ by gradient of the loss function in respect to by adjusting $h_x$.  We can thus treat $h_x$ as parameters and take derivatives:

$$\frac{\partial J}{\partial h_(x)} = h(x) - y $$

Thus, we can interpret residuals as negative gradients:

$$ 
\begin{aligned}
y & = h_0(x) + h_1(x)\\
& = h_0(x) + (y - h_0(x)) \\
& = h_0(x) - (h_0(x) - y) \\
& = h_0(x) - \frac{\partial J}{\partial h_0(x)}
\end{aligned}
$$

So in fact, we are using "gradient" descent in "gradient" boosting to find the new model, written as:

$$h_1(x) = - \frac{\partial J}{\partial h_0(x)} = y - h_0(x)$$

or more generally

$$h_s(x) = - \frac{\partial J}{\partial h_{s-1}(x)} = y - h_{s-1}(x)$$

where $s$ is the index of predictor

**So residuals or gradients?**

Although they are equivalent in the mse loss function, it is more useful to use negative gradients as it is more general, and can apply to other loss functions as well, e.g., **cross-entropy** in the case of classification.

In cross entropy, the loss function is

 $$J= y \lg h(x) + (1 - y) \lg (1-h(x))$$
 
If you look at our previous lecture on logistic regression, the derivative of this **in respect to h(x)** will be:

$$\frac{\partial J}{\partial h_(x)} = h(x) - y$$

This may look the same as mse, but note that our $h(x)$ (i.e., regression tree) outputs continuous values.  In order to transform $h(x)$ into discrete class, we shall transform using sigmoid function $g$ as follows:

$$g(h(x)) = g(z) = \frac{1}{1+e^{-z}}$$

For multiclass classification, $g$ is defined as the softmax function:

$$g(h(x)) = g(z) = \frac{e^z_c}{\displaystyle\sum_{i=1}^{k} e^z_k}$$

Also remind that to use softmax function, we need to first one-hot encode our y.  And during prediction, we need to perform <code>np.argmax</code> along the axis=1


**Adding learning rate**

To make sure adding the subsequent predictor would not overfit our model, we shall add an learning rate $\alpha$ in front of this, which shall be the same across all predictors (different from AdaBoost where alpha is different across all predictors)

$$h_s(x) = - \alpha \frac{\partial J}{\partial h_{s-1}(x)}$$

**What about next predictor**

We can stop if we are happy, either using some predefined iterations, or whether the residual does not decrease further using some validation set.  

In this case, it is obvious that 2 predictors are simply not enough.  Thus, we first need to calculate the residuals which are

$$ residual_1 =  y - (h_0(x) + \alpha h_1(x))$$

then we define $h_2(x)$ as 

$$h_2(x) = \alpha(y - (h_0(x) + \alpha h_1(x)))$$

And then repeat

The final prediction shall use the following hypothesis function $H(x)$:

$$
H(x) = h_0(x) + \alpha_1h_1(x) + \cdots + \alpha_sh_s(x)
$$

**Summary of steps**

1. Initialize the model as simply mean or some constant
2. Predict and calculate the residual
3. Let the next model fit the residual
4. Predict using the combined models and calculate the residual
5. Let the next model fit this residual
6. Simply repeat 4-5 until stopping criteria is reached

### Scratch

In [10]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor

def grad(y, h):
    return y - h

def fit(X, y, models):
    
    models_trained = []
    
    #using DummyRegressor is a good technique for starting model
    first_model = DummyRegressor(strategy='mean')
    first_model.fit(X, y)
    models_trained.append(first_model)
    
    #fit the estimators
    for i, model in enumerate(models):
        #predict using all the weak learners we trained up to
        #this point
        y_pred = predict(X, models_trained)
        
        #errors will be the total errors maded by models_trained
        residual = grad(y, y_pred)
        
        #fit the next model with residual
        model.fit(X, residual)
        
        models_trained.append(model)
        
    return models_trained
        
def predict(X, models):
    learning_rate = 0.1  ##hard code for now
    f0 = models[0].predict(X)  #first use the dummy model
    boosting = sum(learning_rate * model.predict(X) for model in models[1:])
    return f0 + boosting

In [11]:
# Regression

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

n_estimators = 200
tree_params = {'max_depth': 1}
models = [DecisionTreeRegressor(**tree_params) for _ in range(n_estimators)]

#fit the models
models = fit(X_train, y_train, models)

#predict
y_pred = predict(X_test, models)

#print metrics
print("Our MSE: ", mean_squared_error(y_test, y_pred))

Our MSE:  12.945557601580582


### Sklearn 

sklearn has implemented GradientBoosting under the API of <code>GradientBoostingClassifier</code> for classification and <code>GradientBoostingRegressor</code> for regression.

In [7]:
#Compare to sklearn: ls is the same as our mse
sklearn_model = GradientBoostingRegressor(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1,
    loss='ls'
)

y_pred_sk = sklearn_model.fit(X_train, y_train).predict(X_test)

#print metrics
print("Sklearn MSE: ", mean_squared_error(y_test, y_pred_sk))

Sklearn MSE:  12.945557601580584


#### XGBoost

XGBoost is an optimized distributed gradient boosting, designed to be more efficient, flexible, and portable (Chen and Guestrin 2016).  In fact, XGBoost is often an important component of the winning entries in ML competitions (e.g., Kaggle).  XGBoost also offers several nice features, such as automatically taking care of early stopping: XGBoost’s API is quite similar to Scikit-Learn’s:

In [8]:
#make sure to pip install xgboost
#for mac guys, do "brew install libomp" which installs openMP library
#required for XGBoost

import xgboost

xgb_reg = xgboost.XGBRegressor() 

#not improved after 2 iterations
xgb_reg.fit(X_train, y_train,
                eval_set=[(X_test, y_test)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)  #notice we are using mse while xgb uses root mse

[0]	validation_0-rmse:16.15458
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:11.84377
[2]	validation_0-rmse:8.79602
[3]	validation_0-rmse:6.72584
[4]	validation_0-rmse:5.46526
[5]	validation_0-rmse:4.65454
[6]	validation_0-rmse:4.08462
[7]	validation_0-rmse:3.76129
[8]	validation_0-rmse:3.54313
[9]	validation_0-rmse:3.37742
[10]	validation_0-rmse:3.24836
[11]	validation_0-rmse:3.18872
[12]	validation_0-rmse:3.10860
[13]	validation_0-rmse:3.09993
[14]	validation_0-rmse:3.08393
[15]	validation_0-rmse:3.08760
[16]	validation_0-rmse:3.06310
[17]	validation_0-rmse:3.05292
[18]	validation_0-rmse:3.05715
[19]	validation_0-rmse:3.05827
Stopping. Best iteration:
[17]	validation_0-rmse:3.05292

MSE: 9.320308418219375


Let's look at time

In [9]:
%timeit xgboost.XGBRegressor().fit(X_train, y_train)
%timeit GradientBoostingRegressor().fit(X_train, y_train)

30.1 ms ± 886 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
79.6 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### ===Task===

Modify the above scratch code such that:
- Notice that we are still using max_depth = 1.  Attempt to tweak min_samples_split, max_depth for the regression and see whether we can achieve better mse on our boston data
- Notice that we only write scratch code for gradient boosting for regression, add some code so that it also works for binary classification.  Load the breast cancer data from sklearn and see that it works.
- Further change the code so that it works for multiclass classification.  Load the digits data from sklearn and see that it works
- Put everything into class

### When to use Boosting

Let's summarize some useful info about Gradient Boosting:

Advantages:
1. Extremely powerful - especially useful for heterogeneous data (e.g., house price, number of bedrooms). 

Disadvantages:
1. They cannot be parallelized.  Obvious since they are sequential predictors.
2. They can easily overfit, thus require careful choice of estimators or the use of regularization such as max_depth.
3. When we talk about homogeneous data such as images, videos, audio, text, or huge amount of data, deep learning works better.